In [14]:
import parastell.parastell as ps
import parastell.magnet_coils as magnet_coils
import parastell.invessel_build as ivb
import numpy as np
import pystell.read_vmec as read_vmec
from parastell.cubit_io import init_cubit
import cubit
from plotContour import plotContour


init_cubit()


In [2]:


coils_file = 'coils_wistelld.txt'
circ_cross_section = ['circle', 25]
toroidal_extent = 90.0

coils = magnet_coils.MagnetSet(coils_file, circ_cross_section, toroidal_extent)
coils._extract_filaments()
coils._set_average_radial_distance()
coils._set_filtered_filaments()

filtered_filaments = coils.filtered_filaments

coils.build_coil_surface()
coils.export_step('magnet_surface.step')



14:39:07: Exporting STEP file for magnet coils...


In [3]:


# Define directory to export all output files to
export_dir = ''
# Define plasma equilibrium VMEC file
vmec_file = 'plasma_wistelld.nc'

vmec = read_vmec.VMECData(vmec_file)

# Define build parameters for in-vessel components
toroidal_angles = np.linspace(0, 90, 72)
poloidal_angles = np.linspace(0,360,96)
wall_s = 1.08

# Define a matrix of uniform unit thickness
uniform_unit_thickness = np.ones((len(toroidal_angles), len(poloidal_angles)))

radial_build_dict = {}

radial_build = ivb.RadialBuild(toroidal_angles, poloidal_angles, wall_s, radial_build_dict)
build = ivb.InVesselBuild(vmec, radial_build, split_chamber = False)
build.populate_surfaces()
build.calculate_loci()
build._poloidal_angles_exp.shape
build._toroidal_angles_exp.shape

14:39:26: Constructing radial build...
14:39:26: Populating surface objects for in-vessel components...
14:39:29: Computing point cloud for in-vessel components...


(72,)

In [4]:
loci = build.get_loci()[0]
ribs = build.Surfaces['chamber'].Ribs
len(ribs)

72

In [5]:
distances = []
for rib in ribs:
    print(rib)
    distance_subset = []
    for point, direction in zip(rib.rib_loci, rib._normals()):
        cubit.cmd(f"create vertex {point[0]} {point[1]} {point[2]}")
        vertex_id = max(cubit.get_entities("vertex"))
        cubit.cmd(f'create curve location at vertex {vertex_id} location fire ray location at vertex {vertex_id} direction {direction[0]} {direction[1]} {direction[2]} at surface all maximum hits 1')
        curve_id = max(cubit.get_entities('curve'))
        distance = cubit.get_curve_length(curve_id)
        distance_subset.append(distance)
    distances.append(distance_subset)

In [6]:
cubit.cmd('save cub5 "lces.cub5" overwrite')

True

In [8]:
thickness_matrix = np.array(distances)

In [9]:
import pandas as pd 
df = pd.DataFrame(thickness_matrix)
df.to_csv('data2.csv')

min(thickness_matrix.flatten())

106.12068696531642

In [11]:
num_phi = thickness_matrix.shape[0]
num_theta = thickness_matrix.shape[1]

toroidal_angles = np.rad2deg(build._toroidal_angles_exp)
poloidal_angles = np.rad2deg(build._poloidal_angles_exp)

print(toroidal_angles)
print(poloidal_angles)


[ 0.          1.26760563  2.53521127  3.8028169   5.07042254  6.33802817
  7.6056338   8.87323944 10.14084507 11.4084507  12.67605634 13.94366197
 15.21126761 16.47887324 17.74647887 19.01408451 20.28169014 21.54929577
 22.81690141 24.08450704 25.35211268 26.61971831 27.88732394 29.15492958
 30.42253521 31.69014085 32.95774648 34.22535211 35.49295775 36.76056338
 38.02816901 39.29577465 40.56338028 41.83098592 43.09859155 44.36619718
 45.63380282 46.90140845 48.16901408 49.43661972 50.70422535 51.97183099
 53.23943662 54.50704225 55.77464789 57.04225352 58.30985915 59.57746479
 60.84507042 62.11267606 63.38028169 64.64788732 65.91549296 67.18309859
 68.45070423 69.71830986 70.98591549 72.25352113 73.52112676 74.78873239
 76.05633803 77.32394366 78.5915493  79.85915493 81.12676056 82.3943662
 83.66197183 84.92957746 86.1971831  87.46478873 88.73239437 90.        ]
[  0.           3.78947368   7.57894737  11.36842105  15.15789474
  18.94736842  22.73684211  26.52631579  30.31578947  34.1

In [15]:
plotContour(thickness_matrix.T,toroidal_angles,poloidal_angles,'Radial Distance [CM]',
            "phi [deg]", 'theta [deg]', 'SOL to Coils Distance [CM]')